<a href="https://colab.research.google.com/github/FelixBoltz2/CoraClassi/blob/main/CoraClassi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
from google.colab import files
# provide the extracted cora.content file here
uploaded = files.upload()

Saving cora.content to cora (1).content


In [99]:
import pandas as pd
import io
# load the data into a pandas dataframe
content = pd.read_csv('cora.content', sep='\t', header=None)

In [100]:
!pip install scikit-multilearn
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold
import numpy as np

# Splitting the dataset into 10 differents sets in order to use 10-fold cross validation
train_set = []
test_set = []

kf = KFold(n_splits=10, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(content)):
  train_set.append(content.iloc[train_index])
  test_set.append(content.iloc[test_index])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
# define a model using the OneVsRestClassifier since it is a multiclass classification problem and classes are independent
model = OneVsRestClassifier(LinearSVC())

In [109]:
from __future__ import division
# for every one of the 10 different generated sets fit the model, predict the test labels, 
# store the predictions in a tsv-file and evaluate the accuracy
mean_accuracy = 0
predictions = pd.DataFrame({'paper_id':[], 'class_label':[]})

for i in range(10):
  # extract the labels from the dataframe
  train_labels = train_set[i].iloc[:,[1434]]
  test_labels = test_set[i].iloc[:,[1434]]
  # extract the word attributes from the dataframe
  train_word_attributes = train_set[i].iloc[:,1:1434]
  test_word_attributes = test_set[i].iloc[:,1:1434]
  # now the model can be trained
  model.fit(train_word_attributes, train_labels)
  # extract the paper-ids from the dataframe
  prediction_ids = test_set[i].iloc[:,0]
  # let the model predict the labels on the test split
  prediction_values = np.array(model.predict(test_word_attributes), dtype='str')
  output = pd.DataFrame({'paper_id':prediction_ids, 'class_label':prediction_values})
  predictions = pd.concat([predictions, output])
  # evaluate the model on the test split
  print('Fold ' + str(i) + ' accuracy: ' + str(model.score(test_word_attributes, test_labels)))
  mean_accuracy += model.score(test_word_attributes, test_labels)
# store the prediction values and the corresponding paper-ids in a tsv-file
predictions.to_csv('predictions.tsv', sep="\t", index=False)
# calculate overall accuracy (percentage of samples that have their label classified correctly)
mean_accuracy /= 10
print('Overall accuracy is: ' + str(mean_accuracy))

Fold 0 accuracy: 0.7490774907749077
Fold 1 accuracy: 0.7232472324723247
Fold 2 accuracy: 0.7416974169741697
Fold 3 accuracy: 0.7121771217712177
Fold 4 accuracy: 0.7564575645756457
Fold 5 accuracy: 0.7712177121771218
Fold 6 accuracy: 0.7527675276752768
Fold 7 accuracy: 0.7416974169741697
Fold 8 accuracy: 0.7111111111111111
Fold 9 accuracy: 0.7333333333333333
Overall accuracy is: 0.7392783927839277
